# Horner's Method

What is the best way to evaluate 

\\[ f(x) = x^3 + 4x^2 - 10 \\]

at $ x = \dfrac{1}{2} $? The traditional and direct approach is

\\[ f(2) = \dfrac{1}{2} * \dfrac{1}{2} * \dfrac{1}{2} + 4 * \dfrac{1}{2} * \dfrac{1}{2} - 10 \\]

This procedure takes 4 multiplications and 2 additions where a subtraction can be interpreted as adding a negative number. All together it takes 6 operations to evaluate the function. Is there a way to reduce the number of operations? Rewrite the polynomial in such a way the variable $x$ is factored out:
 
\begin{align}
f(x) & = -10 + 4x^2 + x^3 \\
 & = -10 + x * (0 + 4x + x^2 ) \\
 & = -10 + x * (0 + x * (4 + x))
\end{align}

As you can see, it takes a total of 5 operations to evaluate the function; 3 additions and 2 multiplications. This method is called **Horner's Method**.
The point of this notebook is to check the absolute error and relative error and to test the efficiency of using Horner's Method for numerical computations. In this case, we explore its application to find the root of the equation using Bisection Method.

## Absolute and Relative Error
To calculate the error between the two functions we need to find the absolute and the relative error where $x^*$ is the approximate function and $x$ is the true function.

**Absolute Error** $= |x^* - x| $

**Relative Error** $= \dfrac{|x^* - x|}{|x|} $

In [1]:
def error_analysis(true_fx, approx_fx):
    absolute_error = abs(approx_fx - true_fx)
    relative_error = absolute_error / abs(true_fx)
    return absolute_error, relative_error

## True Function - Straightforward Method

In [2]:
def f(x):
    return (x ** 3) + (4 * (x ** 2)) - 10

## Approximate Function - Horner's Method

In [3]:
def f_a(x):
    return -10 + 𝑥 * (0 + 𝑥 * (4 + 𝑥))

## Bisection Method
Bisection method is a root-finding algorithm based from the intermediate-value theorem from calculus.
You need to verify if a root exist by making sure the two endpoints of the interval $ [a,b] $ or $\{f(a),f(b)\}$ have different signs. If function $ f $ is continuous, then there will be a root $r$ between $a$ and $b$ such that $f(r) = 0$ and $a < r < b$.

In [4]:
import numpy as np

In [5]:
def bisection_method(a,b):
    if f(a) * f(b) < 0: 
        i = 0
        roots = []
        while (b - a) / 2 > 1e-7:
            c = (a + b) / 2          
            if f(a) * f(c) < 0:
                b = c
            else:
                a = c
            print(f'{i+1}:\troot = {c:.20f}')
            roots.append(c)
            i += 1
    return np.array(roots)

In [6]:
%%time
roots = bisection_method(1,2)

1:	root = 1.50000000000000000000
2:	root = 1.25000000000000000000
3:	root = 1.37500000000000000000
4:	root = 1.31250000000000000000
5:	root = 1.34375000000000000000
6:	root = 1.35937500000000000000
7:	root = 1.36718750000000000000
8:	root = 1.36328125000000000000
9:	root = 1.36523437500000000000
10:	root = 1.36425781250000000000
11:	root = 1.36474609375000000000
12:	root = 1.36499023437500000000
13:	root = 1.36511230468750000000
14:	root = 1.36517333984375000000
15:	root = 1.36520385742187500000
16:	root = 1.36521911621093750000
17:	root = 1.36522674560546875000
18:	root = 1.36523056030273437500
19:	root = 1.36522865295410156250
20:	root = 1.36522960662841796875
21:	root = 1.36523008346557617188
22:	root = 1.36522984504699707031
23:	root = 1.36522996425628662109
CPU times: user 2.39 ms, sys: 1.52 ms, total: 3.9 ms
Wall time: 3.28 ms


In [7]:
def bisection_method_hm(a,b):
    if f_a(a) * f_a(b) < 0: 
        i = 0
        roots = []
        while (b - a) / 2 > 1e-7:
            c = (a + b) / 2          
            if f_a(a) * f_a(c) < 0:
                b = c
            else:
                a = c
            print(f'{i+1}:\troot = {c:.20f}')
            roots.append(c)
            i += 1
    return np.array(roots)

In [8]:
%%time
roots_a = bisection_method_hm(1,2)

1:	root = 1.50000000000000000000
2:	root = 1.25000000000000000000
3:	root = 1.37500000000000000000
4:	root = 1.31250000000000000000
5:	root = 1.34375000000000000000
6:	root = 1.35937500000000000000
7:	root = 1.36718750000000000000
8:	root = 1.36328125000000000000
9:	root = 1.36523437500000000000
10:	root = 1.36425781250000000000
11:	root = 1.36474609375000000000
12:	root = 1.36499023437500000000
13:	root = 1.36511230468750000000
14:	root = 1.36517333984375000000
15:	root = 1.36520385742187500000
16:	root = 1.36521911621093750000
17:	root = 1.36522674560546875000
18:	root = 1.36523056030273437500
19:	root = 1.36522865295410156250
20:	root = 1.36522960662841796875
21:	root = 1.36523008346557617188
22:	root = 1.36522984504699707031
23:	root = 1.36522996425628662109
CPU times: user 1.09 ms, sys: 637 µs, total: 1.72 ms
Wall time: 1.43 ms
